In [39]:
import ipywidgets as widgets
import anywidget
import traitlets

class ExampleWidget(anywidget.AnyWidget):
    _esm = """
    export function render({ model, el }) {        
        el.classList.add("custom-widget");
        let input = document.createElement('input');
        input.value = 'test';
        input.classList.add("search");
        input.addEventListener("keypress", function(event){ if (event.key === "Enter") {addValue(input.value);} });
        let list = document.createElement('div');

        function addValue(value){
            if(value == ""){
                return;
            }
            for(let index = 0; index < list.children.length; index++){
                if(list.children.item(index).innerHTML == value + '  <span class="close">x</span>'){
                    return;
                }
            }
            let tempText = document.createElement('div');
            tempText.innerHTML = value;
            tempText.classList.add("searchedValue");
            tempText.innerHTML = tempText.innerHTML + '&ensp; <span class="close">x</span>';
            list.appendChild(tempText);
            var closebtns = document.getElementsByClassName("close");

            for (let i = 0; i < closebtns.length; i++) {
              closebtns[i].addEventListener("click", function() {
                this.parentElement.remove();
              });
            }
        }
        
        let widget = document.createElement('div');
        widget.appendChild(input);
        widget.appendChild(list);
        el.appendChild(widget);
    }
    """
    _css = """
    .custom-widget {
        background-color: rgba(0,0,0,0.0);
        padding: 0px 2px;
    }
    .border{
        border: 2px solid;
        border-radius: 25px;
    }
    .search{
        width: 100%;
        outline: 0;
        border: 0;
        border-bottom: 3px solid black;
    }
    .searchedValue {
        background-color: lightgreen;
        padding: 2px 10px;
        margin: 5px 5px;
        border-radius: 25px;
        width: fit-content;
        display: inline-block;
    }
    .close {
      cursor: pointer;
    }
    """
    value = traitlets.Unicode("bob").tag(sync=True)

exampleWidget = ExampleWidget()


display(exampleWidget)

ExampleWidget()